KL-Diversence 적용
- 코로나19 발생 이후, 업종들의 통상적인 시계열 특성이 많이 바꼈다. 그렇기때문에,지역별,업종별, 소비자 정보별 19년 소비 분포와 20년 소비 분포의 차이를 분석해 유사도를 뽑아보는 생각에 착안

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
import gc

import warnings
warnings.filterwarnings(action = 'ignore')

from scipy.spatial import distance
from math import log2
from math import sqrt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [4]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/train_df_noNAN.csv')
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

In [5]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


우선 AMT 0 row를 만들기 위해 groupby() 문법을 활용하여 데이터 프레임을 새롭게 만들어 보자.

In [6]:
st_form = data.groupby(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC',])['AMT'].sum().reset_index().drop('AMT',axis=1).copy()

In [7]:
st_form['REG_YYMM'] = 201901
st_201901=st_form.copy()
st_201901 = st_201901.merge(data[data['REG_YYMM']==201901],
                            on=['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],
                            how='left')


for day in [201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202007]:
  st_form['REG_YYMM'] = day
  st_day = st_form.copy()
  st_day = st_day.merge(data[data['REG_YYMM']==day],
                            on=['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],
                            how='left')
  st_201901 = pd.concat([st_201901,st_day])
  gc.collect()

st_all = st_201901.copy()

In [8]:
st_all.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,REG_YYMM,CSTMR_CNT,AMT,CNT
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,201901,4.0,311200.0,4.0
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,201901,NaN,NaN,NaN
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,201901,7.0,1374500.0,8.0
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,201901,6.0,818700.0,6.0
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,201901,4.0,1717000.0,5.0


In [9]:
st_all.shape

(59165355, 12)

In [10]:
# null 값 확인
st_all.isnull().sum()

CARD_SIDO_NM           0
CARD_CCG_NM            0
STD_CLSS_NM            0
HOM_SIDO_NM            0
HOM_CCG_NM             0
AGE                    0
SEX_CTGO_CD            0
FLC                    0
REG_YYMM               0
CSTMR_CNT       34467563
AMT             34467563
CNT             34467563
dtype: int64

In [11]:
# null 값 0 삽입
st_all.fillna(0,inplace=True)

In [ ]:
# st_all.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/st_all.csv',encoding='utf-8-sig',index=False)

In [12]:
# train, test 쪼개기

train_df = st_all[(st_all['REG_YYMM'] != 202007)&(st_all['REG_YYMM'] != 202004)]
test_df = st_all[(st_all['REG_YYMM'] == 202007&(st_all['REG_YYMM'] == 202004))]

del st_all
gc.collect()

11

---

# Jensen-Shannon Divergence 

## 업종별 AMT 유사도

In [ ]:
job_amt= train_df.groupby(['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['AMT'].sum().reset_index()

In [ ]:
amt_sum = job_amt.groupby(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='AMT_sum')
job_amt = job_amt.merge(amt_sum,on=['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'],how='left')
job_amt['AMT_prob'] = job_amt['AMT']/job_amt['AMT_sum']

# pivot_table을 활용하여 long to wide 작업 실시
amt_pivot= pd.pivot_table(job_amt,
                          index=['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'],
                          columns='REG_YYMM',
                          values='AMT_prob').reset_index()
amt_pivot.fillna(0,inplace=True)

In [ ]:
amt_array=np.array(amt_pivot.drop(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'],axis=1))

job_similarity =[]
for row_r in range(amt_array.shape[0]):
    if row_r%100 ==0 and row_r !=0:
        print(row_r)
        
    dt_list=[]
    for row_c in range(amt_array.shape[0]):
        #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
        '''
        js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
        sqrt(js_divergence)인 distance.jensenshannon 함수 사용
        '''
        dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
        dt_list.append(dt)
    job_similarity.append(dt_list)

similarity_frame =pd.DataFrame(np.array(job_similarity))
job_similarity = pd.concat([amt_pivot[['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM']],similarity_frame],axis=1)j

job_similarity.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/job_similarity.csv',encoding='utf-8-sig',index=False)

## 소비자별 AMT 유사도
- 메모리 과부하때문에 업종별 AMT 유사도는 같이 실행 X (전에 실행완료)


In [ ]:
customer_amt= train_df.groupby(['REG_YYMM','HOM_SIDO_NM','HOM_CCG_NM','AGE', 'SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index()

In [14]:
amt_sum = customer_amt.groupby(['HOM_SIDO_NM','HOM_CCG_NM','AGE', 'SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index(name='AMT_sum')
customer_amt = customer_amt.merge(amt_sum,on=['HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],how='left')
customer_amt['AMT_prob'] = customer_amt['AMT']/customer_amt['AMT_sum']
# pivot_table을 활용하여 long to wide 작업 실시
amt_pivot= pd.pivot_table(customer_amt,
                          index=['HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],
                          columns='REG_YYMM',
                          values='AMT_prob').reset_index()
amt_pivot.fillna(0,inplace=True)

In [ ]:
#피벗된 연도별 amt를 사용해 고객간의 유사도를 구해보자.
amt_array=np.array(amt_pivot.drop(['HOM_SIDO_NM','HOM_CCG_NM','AGE', 'SEX_CTGO_CD','FLC'],axis=1))

customer_similarity =[]
for row_r in range(amt_array.shape[0]):
    if row_r%100 ==0 and row_r !=0:
        print(row_r)
        
    dt_list=[]
    for row_c in range(amt_array.shape[0]):
        #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
        '''
        js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
        sqrt(js_divergence)인 distance.jensenshannon 함수 사용
        '''
        dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
        dt_list.append(dt)
    customer_similarity.append(dt_list)

customer_frame =pd.DataFrame(np.array(customer_similarity))
customer_similarity = pd.concat([amt_pivot,customer_frame],axis=1)

customer_similarity.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/customer_similarity.csv',encoding='utf-8-sig',index=False)